In [ ]:
from ugot import ugot
got = ugot.UGOT()
got.initialize('192.168.1.220')

In [ ]:
running = True

while running:
    direction, prompt = got.start_audio_asr_doa(duration=2)
    # Direction: Left/Right/Front/Behind
    print(f'Direction: {direction} | Prompt: {prompt}')
    got.play_audio_tts(prompt, voice_type=0, wait=True)

    if 'exit' in prompt.lower():
        running = False
        got.play_audio_tts("Exiting robot. Have a nice day!", voice_type=0, wait=True)
        break

    if direction == 'Left':
        got.mecanum_translate_speed_times(angle=-90, speed=30, times=50, unit=1)
    elif direction == "Right":
        got.mecanum_translate_speed_times(angle=90, speed=30, times=50, unit=1)
    elif direction == "Front":
        got.mecanum_translate_speed_times(angle=0, speed=30, times=50, unit=1)
    elif direction == "Back":
        got.mecanum_translate_speed_times(angle=-180, speed=30, times=50, unit=1)

In [ ]:
import re

def parse_command(speech):
    pattern = r"\b(forward|backward|left|right)\s+(\d+(?:\.\d+)?)\b"
    match = re.search(pattern, speech.lower())
    if match:
        return match.group(1), int(match.group(2))
    return None, None

def drive_with_speech():
    print("Voice controlled driving activated. Say 'exit' to stop.")
    while True:
        # Listen for up to 3 seconds
        _, speech = got.start_audio_asr_doa(duration=3)
        print(f"Recognized: {speech!r}")

        if not speech:
            got.play_audio_tts("I didn't hear anything. Please repeat.", voice_type=0, wait=True)
            continue

        # Exit condition
        if any(i in speech.lower() for i in ('exit', 'quit', 'rage quit', 'self destruct')):
            got.play_audio_tts("Shutting down voice control. Goodbye!", voice_type=0, wait=True)
            break

        cmd, dist = parse_command(speech)
        if cmd:
            got.play_audio_tts(f"Moving {cmd} {dist}", voice_type=0, wait=True)
            if cmd == 'left':
                got.mecanum_translate_speed_times(angle=-90, speed=30, times=50, unit=1)
            elif cmd == "right":
                got.mecanum_translate_speed_times(angle=90, speed=30, times=50, unit=1)
            elif cmd == "forward":
                got.mecanum_translate_speed_times(angle=0, speed=30, times=50, unit=1)
            elif cmd == "backward":
                got.mecanum_translate_speed_times(angle=-180, speed=30, times=50, unit=1)
        else:
            got.play_audio_tts("Command not recognized. Please say something like forward 5",
                               voice_type=0, wait=True)

drive_with_speech()

In [ ]:
prompt = got.start_audio_asr()

print(prompt)

got.start_audio_nlp(prompt, wait=True)